In [2]:
import re
import requests
import csv
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
    
# Tole dvoje je treba, da se Chrome sproti ne odpira
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)

C:\Users\jasak\AppData\Local\Temp\ipykernel_25792\2249394969.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)


In [4]:
vzorec_bloka = re.compile(
    r'<td><mtg-set-icon.*?'
    r'</tr',
    flags=re.DOTALL
)

vzorec_karte = re.compile(
    r'<td><mtg-set-icon.*?>.*?<i _ngcontent.*? class="ss ss-fw ss-(?P<set>.+?)\b( ss-.*?)?"></i><!----></mtg-set-icon>'
    r'<a href="/prints/(?P<id_karte>\d*).*?">'
    r'(?P<ime>.*?)'
    r'</a>.*?</td>.*?<td> (?P<redkost>.*?) </td>'
    r'.*?<td class="text-end">'
    r' (?P<povprecna_cena>.*?) </td>(<\!---->)+(<td class="text-end">.*?</td>.*?<td class="text-end"> (?P<povprecna_cena_foil>.*?) </td>.*?<td class="text-end">.*?</td>)?',  
    flags=re.DOTALL
    )

def izloci_podatke_o_kartah(blok):
    karta = vzorec_karte.search(blok).groupdict()
    karta['id_karte'] = int(karta['id_karte'])
    karta['ime'] = karta['ime'].strip()
    karta['set'] = karta['set'].upper()
    karta['redkost'] = karta["redkost"].strip()
    karta['povprecna_cena'] = karta["povprecna_cena"]
    karta['povprecna_cena_foil'] = karta["povprecna_cena_foil"]

    return karta

def pridobi_ustrezno_ime_lokalne_datoteke(st_strani):
    return f"Podatki o kartah/Karte iz seta st. {st_strani}.html"

In [5]:
# Najprej poradiramo csv in json datoteki
with open("karte.csv", "w") as dat:
    pass

with open("karte.json", "w") as dat:
    pass

PRVI_SET = 229
ZADNJI_SET = 231

indeksi_setov_ki_obstajajo = []

for st_strani in range(PRVI_SET, ZADNJI_SET + 1):
    url = (
        f'https://www.mtgstocks.com/sets/{st_strani}' 
    )
    print(f"\n Zajemam {url}\n")
    driver.get(url)
    time.sleep(1)
    
    # response = requests.post(url, allow_redirects=False, timeout=5, headers={
    #     # "Accept-Language": "sl-si"
    # })
    vsebina = driver.page_source
    naslov_strani = driver.title
    
    stevilo_podstrani_s_kartami = vsebina.count("pagination-page page-item")
    st_besed_Market_na_strani = vsebina.count(">Market<")
    
    print("Naslov strani:", naslov_strani)
    
    # with open("pomozna_datoteka_za_pregledovanje_zadnje_vsebine.txt", "w") as dat:
    #     dat.write(vsebina)    
    
    if naslov_strani in ["File not found - MTGStocks"]:
        print(f'\n Stran št. {st_strani} ni dosegljiva.\n')
    else:
        print(f'\n Stran št. {st_strani} JE dosegljiva.\n')
        if "Art Series:" in naslov_strani or "From the Vault:" in naslov_strani or st_besed_Market_na_strani == 0:
            continue
        else:
            indeksi_setov_ki_obstajajo.append(st_strani)
            with open(pridobi_ustrezno_ime_lokalne_datoteke(st_strani), 'w') as dat:
                driver.find_element(by=By.XPATH, value = "//*[text() = 'Market']//..").click()
                for podstran in range(1, stevilo_podstrani_s_kartami + 1):
                    driver.find_element(by=By.XPATH, value = f"//*[contains(@class, 'pagination-page page-item')][{podstran}]//a").click()
                    time.sleep(1)
                    print(f"Pobrana {podstran}. podstran")
                    dat.write(driver.page_source)

# with open("pomozna_datoteka_za_pravilne_indekse.txt", "w") as dat:
#     dat.write(indeksi_setov_ki_obstajajo)  


 Zajemam https://www.mtgstocks.com/sets/229

Naslov strani: Game Day Promos - MTGStocks

 Stran št. 229 JE dosegljiva.

Pobrana 1. podstran
Pobrana 2. podstran

 Zajemam https://www.mtgstocks.com/sets/230

Naslov strani: Duel Decks: Jace vs. Vraska - MTGStocks

 Stran št. 230 JE dosegljiva.

Pobrana 1. podstran
Pobrana 2. podstran

 Zajemam https://www.mtgstocks.com/sets/231

Naslov strani: Magic Player Rewards - MTGStocks

 Stran št. 231 JE dosegljiva.

Pobrana 1. podstran
Pobrana 2. podstran


Koda za zapisovanje kart iz lokalnih datotek v csv in json format.

In [13]:
debug_mode_za_branje_iz_lokalnih_datotek = False
karte = []

for st_seta in indeksi_setov_ki_obstajajo:
    
    stetje = 0
    with open(f"Podatki o kartah/Karte iz seta st. {st_seta}.html", "r") as f:
        print(f"Berem podatke kart iz seta #{st_seta}")
        
        
        vsebina = f.read()
        
        for blok in vzorec_bloka.finditer(vsebina):
            stetje += 1
            
            if debug_mode_za_branje_iz_lokalnih_datotek:
                print(stetje)
                print(blok.group(0))
                print(vzorec_karte.search(blok.group(0)).groupdict())
            
            karte.append(izloci_podatke_o_kartah(blok.group(0)))

with open("karte.json", "a") as dat:
    json.dump(karte, dat, indent=4, ensure_ascii=False)
    

with open("karte.csv", "a") as dat:
    writer = csv.DictWriter(dat, [
        "id_karte",
        "ime",
        "set",
        "redkost",
        "povprecna_cena",
        "povprecna_cena_foil",
    ])
    writer.writeheader()
    writer.writerows(karte)

Berem podatke kart iz seta #229
Berem podatke kart iz seta #230
Berem podatke kart iz seta #231


Zdaj se znebimo kart, ki so tokeni ali basic landi
Prav tako bi se radi znebili kart, ki imajo tako ceno kot ceno foila ničelno. (Torej da lahko karte primerjamo po vsaj eni komponenti)
Pri tem je vredno omeniti, da imajo karte, ki so najbolj zaželene, tudi najbolj popolne podatke, torej jih s tem ne bomo izgubili

In [12]:
podatki_o_kartah = pd.read_csv("karte.csv", encoding="latin1")

regex_izraz_za_basic_lande = r'Forest \(\d|Mountain \(\d|Swamp \(\d|Plains \(\d|Island \(\d|Wastes \(\d'
okrajsani_podatki = podatki_o_kartah[(podatki_o_kartah.redkost != "Token") & ~(podatki_o_kartah.ime.str.match(regex_izraz_za_basic_lande)) & ~(podatki_o_kartah.ime.isin(["Forest", "Mountain", "Swamp", "Plains", "Island", "Wastes"]))]
okrajsani_podatki = okrajsani_podatki[~(okrajsani_podatki.povprecna_cena.isnull() & okrajsani_podatki.povprecna_cena_foil.isnull())]
print(okrajsani_podatki)

     id_karte                     ime    set         redkost povprecna_cena  \
3       83765  Touch the Spirit Realm  PMTG1  Player Rewards          $3.25   
4       25484        Reclamation Sage  PMTG1        Uncommon          $2.75   
6       73402                Consider  PMTG1          Common          $2.47   
7       79012            Spell Pierce  PMTG1  Player Rewards          $1.85   
9       35236                  Abrade  PMTG1        Uncommon          $2.47   
..        ...                     ...    ...             ...            ...   
249     24055          Lightning Bolt  PMTG1            Rare         $29.41   
250     24066              Powder Keg  PMTG1            Rare         $18.07   
251     24068               Psychatog  PMTG1            Rare         $10.43   
252     24091               Wasteland  PMTG1        Uncommon        $323.99   
253     24092            Wrath of God  PMTG1            Rare         $23.56   

    povprecna_cena_foil  
3                   NaN  